In [1]:
import os
import sklearn
import pandas as pd
import csv

In [2]:
n_gram_value = 5

In [3]:
def get_ngrams(input_list, n):    
    return zip(*[input_list[i:] for i in range(n)])


In [4]:
def getAllPossibleWords(file_path): 
    possible_words=[]
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                possible_words.append(word)     
   # possible_words = map(lambda x : unidecode.unidecode(x),possible_words)
    for c in ['`',"[","]","(",")","\"",".","''",","]:
        possible_words = map(lambda x: x.replace(c,""),possible_words)
        
    possible_words = map(lambda x: x.replace('&','And'),possible_words)
    #possible_words = map(lambda x: x.replace('of','Of'),possible_words)
    possible_words=filter(lambda x:len(x)>0,possible_words)    
    
    return possible_words

In [5]:
def getListFromFiles(filename):
    file_path = os.path.join(os.getcwd(),filename)
    info_list = []
    with open(file_path) as f:
        for line in f:
            for word in line.split():
                info_list.append(word)
    return info_list

In [6]:
def writeToCsv(processed_list):
    with open("output-sreejita.csv",'wb') as resultFile:
        wr = csv.writer(resultFile, dialect='excel')
        #for row in processed_list:
        wr.writerows(processed_list)

In [7]:
'''def preprocessing(possible_words,doc_id):    
    suffix_list = getListFromFiles("company-suffixes.txt")
    prefix_list = getListFromFiles("company-prefixes.txt")    
    common_words = getListFromFiles("common-words.txt")
    preprocessed_list =[]    
    marked_list=[]
    word_number=len(possible_words)
    for i in range(5,0,-1):
        word_count=0
        ngram_list = get_ngrams(possible_words,i)
        for ngram in ngram_list:            
            #print ngram   
            
            #if markup is both at the beginning or at the end of the word group accept and label as 1
            if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])
            
            #get all instances of format <markup>Microsoft</markup>
            if(len(ngram)==1 and '</markup>' in ngram[0] and '<markup>' in ngram[0]):
                preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,1])            
                    
            else:
                #prune away all n grams with non uppercase first character
                if(all(word[0].isupper() and "markup" not in word for word in ngram)):       
                    
                    # prune common words
                    if(any(word not in common_words for word in ngram)):                        
                        preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])

                    #prune unigrams 
                    elif(i==1 and word_number> word_count+1):                        
                        
                        #add the unigram to the list if the next word begins with lower case
                        if(possible_words[word_count+1][0].islower()):
                            preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])  
            word_count = word_count+1
    return preprocessed_list   ''' 

'def preprocessing(possible_words,doc_id):    \n    suffix_list = getListFromFiles("company-suffixes.txt")\n    prefix_list = getListFromFiles("company-prefixes.txt")    \n    common_words = getListFromFiles("common-words.txt")\n    preprocessed_list =[]    \n    marked_list=[]\n    word_number=len(possible_words)\n    for i in range(5,0,-1):\n        word_count=0\n        ngram_list = get_ngrams(possible_words,i)\n        for ngram in ngram_list:            \n            #print ngram   \n            \n            #if markup is both at the beginning or at the end of the word group accept and label as 1\n            if(len(ngram)>1 and \'<markup>\' in ngram[0] and \'</markup>\' in ngram[len(ngram)-1] and\n              all((word[0].isupper() or "markup" in word) for word in ngram) and \n              ["<markup>" in word for word in ngram].count(True)==1):\n                preprocessed_list.append([\' \'.join(ngram),doc_id,word_count,word_count+i-1,1])\n            \n            #get all

In [8]:
def preprocessing(possible_words,doc_id):    
    suffix_list = getListFromFiles("company-suffixes.txt")
    prefix_list = getListFromFiles("company-prefixes.txt")    
    common_words = getListFromFiles("common-words.txt")
    preprocessed_list =[]    
    marked_list=[]
    word_number=len(possible_words)
    for i in range(5,0,-1):
        word_count=0
        ngram_list = get_ngrams(possible_words,i)
        for ngram in ngram_list:            
            #print ngram   
            
            #if markup is both at the beginning or at the end of the word group accept and label as 1
            if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1): 
                company_tuple = ngram
                for string in ["<markup>","</markup>"]:
                    company_tuple = map(lambda x : x.replace(string,""),company_tuple)                
                preprocessed_list.append([' '.join(company_tuple),doc_id,word_count,word_count+i-1,1])
            
            #get all instances of format <markup>Microsoft</markup>
            elif(len(ngram)==1 and '</markup>' in ngram[0] and '<markup>' in ngram[0]):
                
                company_tuple = ngram
                for string in ["<markup>","</markup>"]:
                    company_tuple = map(lambda x : x.replace(string,""),company_tuple) 
                
                preprocessed_list.append([' '.join(company_tuple),doc_id,word_count,word_count+i-1,1])            
                    
            else:
                #prune away all n grams with non uppercase first character
                if(all(word[0].isupper() and "markup" not in word for word in ngram)):  
                    
                    #prune unigrams 
                    if(i==1):                        
                        if(ngram[0] not in suffix_list):
                            if(word_number> word_count+1 and ngram[0] not in common_words):
                                #add the unigram to the list if the next word begins with lower case
                                if(possible_words[word_count+1][0].islower()):
                                    preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])
                    
                    #prune common words
                    elif(any(word not in common_words for word in ngram)):                        
                        preprocessed_list.append([' '.join(ngram),doc_id,word_count,word_count+i-1,0])                   

                    
                     
            word_count = word_count+1
    return preprocessed_list    

In [9]:
def generateFeatures(processed_list,possible_words):
    
    company_suffixes = getListFromFiles("company-suffixes.txt")
    company_prefixes = getListFromFiles("company-prefixes.txt")    
    company_list=[]
    
    for company in processed_list:
    
        #Feature 1 : name has any prefix or suffix of a company 
        #if company name has any one of the suffixes or prefixes set it to true 
        hasCompanyid=0        
        if(company[0].split()[-1] in company_suffixes or company[0].split()[0] in company_prefixes):
            hasCompanyid = 1
            company_list.append(company[0])            
        #company.append(hasCompanyid)
        
        
        #has Company name at the beginning ?
        hasCompanyNameFirst = 0
        l1= [x for x in company[0].split() if x not in company_prefixes]
        l2=[]
        for comp in company_list:    
            if(comp.split()[0] in company_prefixes):
                l2.append(" ".join(comp.split()[1:]))
            else:
                l2.append(comp)
        if(any(word in l2 for word in l1)):
            hasCompanyNameFirst = 1
        elif(any(company[0].split()[0]== word.split()[0] for word in company_list)):
            hasCompanyNameFirst = 1 
        
            
        
        #has Company name as a substring ?
        hasNameSubstring=0
        for comp in company_list:
            l1 = company[0].split()
            l2 = comp.split()
            if(len(set(l1)&set(l2)) > 0 ):
                hasNameSubstring=1       
       
        company.extend([hasCompanyid,hasCompanyNameFirst,hasNameSubstring])    

In [10]:
base_directory = os.path.dirname(os.getcwd())
data_directory = os.path.join(base_directory,"Dataset","Testing")
file_list = os.listdir(data_directory)
possible_words=[]
processed_list=[]
count=0
#doc_id = 101
for file in file_list:
    doc_id = file[0:3]
    #print(doc_id)
    #if(int(doc_id)!=95):
    #    continue    
    possible_words = getAllPossibleWords(os.path.join(data_directory,file))     
    
    #start from here .. processessed list has the structure[[word,doc_id,word_count_start,word_count_end,label]]
    ''''processed_list.append(preprocessing(possible_words,doc_id))
    writeToCsv(processed_list)'''
    processed_list_for_doc = preprocessing(possible_words,doc_id)
    print processed_list_for_doc
    generateFeatures(processed_list_for_doc,possible_words)
    processed_list.extend(processed_list_for_doc)
    #writeToCsv(processed_list) 
  

[["National Football League's American Conference", '000', 70, 74, 0], ["National Football League's American", '000', 70, 73, 0], ["Football League's American Conference", '000', 71, 74, 0], ['Nielsen Media Research', '000', 39, 41, 0], ["National Football League's", '000', 70, 72, 0], ["Football League's American", '000', 71, 73, 0], ["League's American Conference", '000', 72, 74, 0], ['Katz Television Group', '000', 142, 144, 1], ['New York A', '000', 146, 148, 0], ['American Idol', '000', 0, 1, 0], ['Nielsen Media', '000', 39, 40, 0], ['Media Research', '000', 40, 41, 0], ['National Football', '000', 70, 71, 0], ["Football League's", '000', 71, 72, 0], ["League's American", '000', 72, 73, 0], ['American Conference', '000', 73, 74, 0], ['Simon Cowell', '000', 113, 114, 0], ['Randy Jackson', '000', 116, 117, 0], ['Bill Carroll', '000', 132, 133, 0], ['New York', '000', 146, 147, 0], ['York A', '000', 147, 148, 0], ['Idol', '000', 1, 1, 0], ['Research', '000', 41, 41, 0], ['Idol', '000

[['Chief Executive Officer John Fletcher', '028', 35, 39, 0], ['Kohlberg Kravis Roberts And Co', '028', 87, 91, 1], ['Chief Executive Officer John', '028', 35, 38, 0], ['Executive Officer John Fletcher', '028', 36, 39, 0], ['Chief Executive Officer', '028', 35, 37, 0], ['Executive Officer John', '028', 36, 38, 0], ['Officer John Fletcher', '028', 37, 39, 0], ['Kravis Roberts And', '028', 88, 90, 0], ['Chairman Rick Allert', '028', 123, 125, 0], ['White Funds Management', '028', 160, 162, 1], ['Chief Executive', '028', 35, 36, 0], ['Executive Officer', '028', 36, 37, 0], ['Officer John', '028', 37, 38, 0], ['John Fletcher', '028', 38, 39, 0], ['Coles Myer', '028', 57, 58, 1], ['Kravis Roberts', '028', 88, 89, 0], ['Roberts And', '028', 89, 90, 0], ['Solomon Lew', '028', 92, 93, 0], ['Coles Myer', '028', 121, 122, 1], ['Chairman Rick', '028', 123, 124, 0], ['Rick Allert', '028', 124, 125, 0], ['Atul Lele', '028', 149, 150, 0], ['Coles Myer', '028', 166, 167, 1], ['Coles Myer', '028', 204

[["After North Korea's Oct", '050', 129, 132, 0], ["UN Resolution North Korea's", '050', 213, 216, 0], ['United Nations Security Council', '050', 243, 246, 0], ['President George W Bush', '050', 250, 253, 0], ["Government Accountability Office Congress's", '050', 401, 404, 0], ['Homeland Security At', '050', 73, 75, 0], ["After North Korea's", '050', 129, 131, 0], ["North Korea's Oct", '050', 130, 132, 0], ['State Condoleezza Rice', '050', 139, 141, 0], ['Civitas Group LLC', '050', 180, 182, 1], ['UN Resolution North', '050', 213, 215, 0], ["Resolution North Korea's", '050', 214, 216, 0], ['United Nations Security', '050', 243, 245, 0], ['Nations Security Council', '050', 244, 246, 0], ['President George W', '050', 250, 252, 0], ['George W Bush', '050', 251, 253, 0], ['North Korea China', '050', 335, 337, 0], ['Plastic Scintillation Most', '050', 350, 352, 0], ['Government Accountability Office', '050', 401, 403, 0], ["Accountability Office Congress's", '050', 402, 404, 0], ['SAI Longm

[['September Stellenbosch South Africa-based', '078', 50, 53, 0], ['British American Tobacco Plc', '078', 180, 183, 1], ['September Stellenbosch South', '078', 50, 52, 0], ['Stellenbosch South Africa-based', '078', 51, 53, 0], ['Absa Group Ltd', '078', 98, 100, 1], ['Sage Group Ltd', '078', 116, 118, 1], ['Momentum Group Ltd', '078', 122, 124, 1], ['Currency Boost Shares', '078', 195, 197, 0], ['Remgro Ltd', '078', 0, 1, 1], ['South African', '078', 4, 5, 0], ['September Stellenbosch', '078', 50, 51, 0], ['Stellenbosch South', '078', 51, 52, 0], ['South Africa-based', '078', 52, 53, 0], ['Barclays Plc', '078', 102, 103, 1], ['American Tobacco', '078', 181, 182, 0], ['Currency Boost', '078', 195, 196, 0], ['Boost Shares', '078', 196, 197, 0], ['REM', '078', 2, 2, 0], ['African', '078', 5, 5, 0], ['Net', '078', 31, 31, 0], ['Remgro', '078', 54, 54, 1], ["That's", '078', 61, 61, 0], ['Last', '078', 77, 77, 0], ["UK's", '078', 105, 105, 0], ['Remgro', '078', 125, 125, 1], ['Net', '078', 15

[['Chief Executive Officer Michael Callen', '206', 62, 66, 0], ['Executive Officer Michael Callen Chief', '206', 63, 67, 0], ['Officer Michael Callen Chief Financial', '206', 64, 68, 0], ['Michael Callen Chief Financial Officer', '206', 65, 69, 0], ['Callen Chief Financial Officer Sean', '206', 66, 70, 0], ['Chief Financial Officer Sean T', '206', 67, 71, 0], ['Financial Officer Sean T Leonard', '206', 68, 72, 0], ['Ambac Financial Group Inc', '206', 0, 3, 1], ['New York Stock Exchange', '206', 43, 46, 0], ['Chief Executive Officer Michael', '206', 62, 65, 0], ['Executive Officer Michael Callen', '206', 63, 66, 0], ['Officer Michael Callen Chief', '206', 64, 67, 0], ['Michael Callen Chief Financial', '206', 65, 68, 0], ['Callen Chief Financial Officer', '206', 66, 69, 0], ['Chief Financial Officer Sean', '206', 67, 70, 0], ['Financial Officer Sean T', '206', 68, 71, 0], ['Officer Sean T Leonard', '206', 69, 72, 0], ['New York Stock', '206', 43, 45, 0], ['York Stock Exchange', '206', 44

[['Empire State Building', '232', 15, 17, 0], ['New York Instead', '232', 23, 25, 0], ['New York City', '232', 102, 104, 0], ['Western Europeans The', '232', 109, 111, 0], ['NYC And Co', '232', 287, 289, 1], ['Bronagh Cullinan', '232', 0, 1, 0], ['Empire State', '232', 15, 16, 0], ['State Building', '232', 16, 17, 0], ['New York', '232', 23, 24, 0], ['York Instead', '232', 24, 25, 0], ['Manolo Blahnik', '232', 95, 96, 1], ['New York', '232', 102, 103, 0], ['York City', '232', 103, 104, 0], ['Western Europeans', '232', 109, 110, 0], ['Europeans The', '232', 110, 111, 0], ['Wall Street', '232', 131, 132, 0], ['George Malkemus', '232', 164, 165, 0], ['Manolo Blahnik', '232', 171, 172, 1], ['Fifth Avenue', '232', 235, 236, 0], ['Madison Avenue', '232', 238, 239, 0], ['Michaela Gilles', '232', 241, 242, 0], ['Duesseldorf Germany', '232', 252, 253, 0], ['FAO Schwarz', '232', 260, 261, 1], ['Cullinan', '232', 1, 1, 0], ['Ireland', '232', 6, 6, 0], ['Statue', '232', 10, 10, 0], ['Liberty', '23

[['Ambac Financial Group Inc', '258', 4, 7, 1], ['Security Capital Assurance Ltd', '258', 122, 125, 1], ['New York Stock Exchange', '258', 178, 181, 0], ['Assured Guaranty Ltd', '258', 117, 119, 1], ['New York Stock', '258', 178, 180, 0], ['York Stock Exchange', '258', 179, 181, 0], ['MBIA Inc', '258', 0, 1, 1], ['Financial Group', '258', 5, 6, 0], ['Calyon Securities', '258', 11, 12, 1], ['Harry Fong', '258', 55, 56, 0], ['The Calyon', '258', 112, 113, 0], ['Capital Assurance', '258', 123, 124, 0], ['Security Capital', '258', 156, 157, 1], ['New York', '258', 178, 179, 0], ['York Stock', '258', 179, 180, 0], ['Stock Exchange', '258', 180, 181, 0], ['Assured Guaranty', '258', 192, 193, 1], ['Security Capital', '258', 201, 202, 1], ['MBI', '258', 2, 2, 0], ['ABKFQ', '258', 8, 8, 0], ['Fong', '258', 56, 56, 0], ['All', '258', 62, 62, 0], ['Bond', '258', 71, 71, 0], ['MBIA', '258', 92, 92, 1], ['Ambac', '258', 105, 105, 1], ['Calyon', '258', 113, 113, 0], ['AGO', '258', 120, 120, 0], ['MB

[['Controladora Comericial Mexicana SAB', '286', 116, 119, 1], ['Controladora Comercial Mexicana', '286', 122, 124, 1], ['Soriana SAB', '286', 1, 2, 1], ['SORIANAB Mexico', '286', 3, 4, 0], ['Gigante SAB', '286', 11, 12, 1], ['Pedro Mejia', '286', 40, 41, 0], ['Comericial Mexicana', '286', 117, 118, 0], ['Mexico City', '286', 137, 138, 0], ['Soriana', '286', 23, 23, 1], ['Gigante', '286', 37, 37, 1], ['Mejia', '286', 41, 41, 0], ['Mexico', '286', 56, 56, 0], ['US', '286', 60, 60, 0], ['Soriana', '286', 68, 68, 1], ['Mexican', '286', 76, 76, 0], ['Soriana', '286', 84, 84, 1], ['Mexico', '286', 90, 90, 0], ['Soriana', '286', 100, 100, 1], ['Mart', '286', 107, 107, 0], ['City', '286', 138, 138, 0], ['Excelsior', '286', 140, 140, 0], ['Walmex', '286', 150, 150, 1], ['Last', '286', 162, 162, 0], ['Soriana', '286', 164, 164, 1], ['Gigante', '286', 172, 172, 1], ['Walmex', '286', 179, 179, 1]]
[['Chief Executive Officer Frank McAllister', '287', 158, 162, 0], ['Chief Executive Officer Frank',

In [547]:
ngram = ('<markup>Gazprom', 'Lukoil</markup>')
if(len(ngram)>1 and '<markup>' in ngram[0] and '</markup>' in ngram[len(ngram)-1] and
              all((word[0].isupper() or "markup" in word) for word in ngram) and 
              ["<markup>" in word for word in ngram].count(True)==1):
    print "hallo"


hallo


In [548]:
["markup" in word for word in ngram].count(True)==2

True

In [549]:
import pycountry
country_names = []
for country in pycountry.countries:
    country_names.append(country.name)
    if hasattr(country,'common_name'):
        country_names.append(country.common_name)
    if hasattr(country, 'official_name'):
        country_names.append(country.official_name)

country_names.append("America")
country_names.append("Korea")
country_names.append("South Korea")
country_names.append("North Korea")
country_names.append("U.S.")
country_names.append("Russia")

if "Russia" in country_names:
    print "YES"
    
continent_list = ['Asia', 'Africa', 'South America', 'North America', 'Antarctica', 'Europe', 'Australia']

YES


In [550]:
cities = []
with open('world-cities.csv', 'r') as inp:
        for row in csv.reader(inp):
            if(row and row[3] != "" and row[0]!="name"):
                cities.append(row[0])

cities.append('Bombay')
cities.append('New York')
if "Compton" in cities:
    print "YES"

us_states = open('us-states.txt').read().split('\n')
ethni_list = open('ethnicities.txt').read().split('\n')

YES


In [551]:
desig = open('designations.txt').read().split('\n')
desig_lower = [x.lower() for x in desig]

In [552]:
pruned_pos = open('pruned_pos.txt').read().split('\n')
stock_x = open('stock-exchange-list.txt').read().split('\n')
unigram_prune = open('unigram-prune-neg.txt').read().split('\n')
special_char = [';', ',', ':', '?']
prune = open('pruning-neg.txt').read().split('\n') 
prune_lower = [x.lower() for x in prune]
suffixes = open('company-suffixes.txt').read().split('\n')
prefixes = open('company-prefixes.txt').read().split('\n')
rows = 0
print len(processed_list)
for doc in processed_list:
    rows += len(doc)


8115


In [554]:
print pruned_pos
new_proc_list = []
pruned_list = []

for inst in processed_list:
    if inst[0] not in pruned_pos: 
        if inst[0] in country_names or inst[0] in continent_list or inst[0] in cities or inst[0] in stock_x or inst[0] in us_states:
            pruned_list.append(inst)
        elif "'s" in inst[0]:
            pruned_list.append(inst)
        elif inst[0].endswith('n') and inst[0][:-1] in country_names or str in continent_list:
            pruned_list.append(inst)
        elif "based" in inst[0]:
            pruned_list.append(inst)
        elif any(d in inst[0].lower().split() for d in desig_lower):
            pruned_list.append(inst)
        elif any(p in inst[0].lower().split() for p in prune_lower):
            pruned_list.append(inst)
        elif '(' in inst[0] or ')' in inst[0]:
            pruned_list.append(inst)
        elif any(c in inst[0] for c in special_char):
            pruned_list.append(inst)
        elif len(inst[0].split()) == 1:
            if len(inst[0]) == 1:
                pruned_list.append(inst)
            elif inst[0] in unigram_prune or inst[0] in ethni_list:
                pruned_list.append(inst)
            elif inst[0] == inst[0].upper():
                pruned_list.append(inst)
            elif inst[0] in suffixes or inst[0] in prefixes:
                pruned_list.append(inst)
            else:
                new_proc_list.append(inst)
        else:
            new_proc_list.append(inst)
    else:
        new_proc_list.append(inst)
            
                
print len(pruned_list)
for eg in pruned_list:
    if eg[4] == 1:
        print "HOLY MOLY!"
#writeToCsv(processed_list)
print len(new_proc_list)
print len(pruned_list) + len(new_proc_list)
#print pruned_list



['ACE', "Sam's Club", 'ICBC', 'KKR', 'CSN', 'Tata', 'Wheeling', "River's Edge Pharmaceuticals LLC", "River's Edge", 'RWE', 'Fremont', 'News Corp', 'GM', 'Liberty', 'Financial Services Authority', 'FSA', 'NZOG', 'Norilsk', 'NTP', 'MPC', 'PPR', 'THQ', 'Nokia', 'RCS', 'Intercontinental Exchange Inc', 'EON', 'RHB', 'WCM', 'Macclesfield', 'Carrefour', 'KPI', 'TXU', 'ECI', 'BP', 'TNK-BP', 'Imperial', 'Zug', 'TDF', 'TMK', "Ivernia Inc's", 'SAIC', 'GE', 'PICC', 'Saku', 'Winterthur', 'ZKB', "Rupert Murdoch's News Corp", 'PIK', 'VTB', 'UBS', 'IGT', 'BHP', 'BBH', 'China Banking Regulatory Commission', 'Compton', 'Centennial', 'NPD', 'HSBC', "Victoria's Secret", 'MAN', 'MBIA', 'Freeport', 'MGIC', 'PMI', 'Stillwater', "Dr Reddy's Laboratories Ltd", "Royal Mail's", '']
2914
5201
8115


In [555]:
'''thefile = open('pruned_pos.txt', 'w')
for item in pruned_pos:
    #print item
    thefile.write("%s\n" % item)
len(pruned_pos)
print pruned_list'''

'thefile = open(\'pruned_pos.txt\', \'w\')\nfor item in pruned_pos:\n    #print item\n    thefile.write("%s\n" % item)\nlen(pruned_pos)\nprint pruned_list'

In [556]:
with open("test_pruned.csv", "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in new_proc_list:
        writer.writerow(val)